## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

Add our `utils` directory to the system's `$PATH` so we can import Python files from sibling directories.

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import re
import json
import pytz
import glob
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup

/Users/slee/.local/share/virtualenvs/california-coronavirus-scrapers-QTClc1WQ/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Download

Retrieve the page

In [6]:
url = "https://infogram.com/1pe66wmyjnmvkrhm66x9362kp3al60r57ex"

In [7]:
page = requests.get(url)

## Parse

In [8]:
soup = BeautifulSoup(page.content, "html.parser")

Find script tag with the Infogram data

In [9]:
for elem in soup(text=re.compile(r"window.infographicData")):
    script = elem

In [10]:
stripped_script = script.replace("window.infographicData=", "")
stripped_script = stripped_script.rstrip(";")

In [11]:
data = json.loads(stripped_script)

In [17]:
cities = data["elements"]["content"]["content"]["entities"]
# [
#     "b26b9acd-b036-40bc-bbbe-68667dd338e4"
# ]["props"]["chartData"]["data"][0]

In [18]:
cities

{'0709e3f0-07d2-4964-94c9-dd32e71c1c938166d4e1-9485-4fa8-ac4a-a9251a9a5d77': {'filters': {},
  'height': 183.3386134999372,
  'left': 2320,
  'lockAspectRatio': True,
  'locked': False,
  'maxHeight': 10000,
  'maxWidth': 10000,
  'minHeight': 1,
  'minWidth': 1,
  'props': {'assetId': '0a71f6d1-095e-441d-a6ff-30cfcfdda37c',
   'graphicType': 'uploaded_image',
   'id': 84306992,
   'opacity': 1,
   'subType': 'image'},
  'top': 0,
  'transform': {'flipHorizontal': False, 'flipVertical': False, 'rotate': 0},
  'type': 'IMAGE',
  'width': 183.3386134999372},
 '0b43bd58-411e-456d-8502-51a4ae1a330420e9c946-7b89-438b-a91f-291116777f2e': {'filters': {},
  'height': 183.3386134999372,
  'left': 2305,
  'lockAspectRatio': True,
  'locked': False,
  'maxHeight': 10000,
  'maxWidth': 10000,
  'minHeight': 1,
  'minWidth': 1,
  'props': {'assetId': '0a71f6d1-095e-441d-a6ff-30cfcfdda37c',
   'graphicType': 'uploaded_image',
   'id': 84306992,
   'opacity': 1,
   'subType': 'image'},
  'top': 11,
 

In [ ]:
not_empty = [ele for ele in cities if ele != [""]]

Convert to dataframe

In [ ]:
df = pd.DataFrame(not_empty, columns=["area", "confirmed_cases"])

Get timestamp

In [ ]:
timestamp = data["updatedAt"]

In [ ]:
latest_date = pd.to_datetime(timestamp).date()

In [ ]:
df.insert(0, "county", "Butte")

In [ ]:
df["county_date"] = latest_date

In [ ]:
df = df[df.area != "Region"]

## Vet

In [ ]:
try:
    assert not len(df) > 6
except AssertionError:
    raise AssertionError("Butte County's city scraper has additional rows")

In [ ]:
try:
    assert not len(df) < 6
except AssertionError:
    raise AssertionError("Butte's city scraper is missing rows")

## Export

Mark the current date

In [ ]:
tz = pytz.timezone("America/Los_Angeles")

In [ ]:
today = datetime.now(tz).date()

In [ ]:
slug = "butte"

In [ ]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [ ]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [ ]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [ ]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [ ]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)